In [2]:
%matplotlib qt5
import numpy as np
from numpy import linalg as la
import matplotlib.pylab as plt
from scipy.optimize import fsolve
import math
import statsmodels.api as sm
from scipy.stats import norm
import seaborn as sns
from scipy import stats
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from Meanfield_Dyn_util import *
from utils import *

In [3]:
import scipy
from functools import partial
def odeIntegral(x,t,J,I=0):
    x = np.squeeze(x)
    x = np.reshape(x,(len(x),1))
    # dxdt = -x+J@np.tanh(x)#+I[0]
    dxdt = -x+J@x+I[0]
    return np.squeeze(dxdt)
def odesimulation(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegral,J=Jpt,I=I),xinit,t)
shiftx = 1.5
def odeIntegralP(x,t,J,I=0):
	x = np.squeeze(x)
	x = np.reshape(x,(len(x),1))
	# print('size:',np.shape(x),np.shape(J@np.tanh(x)))
	dxdt = -x+J@(1.0+np.tanh(x-shiftx))
	return np.squeeze(dxdt)
def odesimulationP(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegralP,J=Jpt,I=I),xinit,t)

In [4]:
### define the network parameters of the adjacency matrix
N  = 1500
Kt = int(N*0.2)
J = 1/np.sqrt(N)*0.5  ### TODO: make sure this scalar with David&Stefano's paper
J = 0.00325
ntau   = 10
trials = 3#30+6
tau_series = np.linspace(0,0.15,ntau)#np.linspace(0.1,0.2,ntau)#
### EEII; EEEIII; EEEIII;EEII
switch = 1
## arrays to store results
## norml0_series: norm of left eigenvector(deltaliri = 1)
eigvchn_series, eigrvec_series, eiglvec_series = np.zeros((trials,ntau,N),dtype=complex), np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
eiglvec0_series, norml0_series = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau+1,N)) 
### also have the reconstructed left and right eigenvectors 
eigrvec_series_rec, eiglvec_series_rec = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
htau = tau_series[1]-tau_series[0]
### simulation using the low-rank framework
firing_rateeq = np.zeros((trials,ntau,N))
### recording dynamics
lowrank_eq, lowrank_eq_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,N))
ovs_inplr, ovs_inplr_num   = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
ovs_inplr_div, ovs_inplr_div_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
contributions_lr, contributions_lr_num = np.zeros((trials,ntau,2,2)), np.zeros((trials,ntau,2,2))### rank, population

## numerical 
intg_ov_series    = np.zeros((trials,ntau,2))
first_perturb_ov  = np.zeros((trials,ntau,2,2))
first_perturb_ovP = np.zeros((trials,ntau,2,2,2)) ### rank and population

intg_mean_series  = np.zeros((trials,ntau,N,2,2))### rank2 and population2
intg_std_series   = np.zeros((trials,ntau,2,2))### rank, population
intg_std_num_series = np.zeros((trials,ntau,2,2))### rank, population
mean_shift_ov     = np.zeros((trials,ntau,2))
latent_kappa_series = np.zeros((trials,ntau,2,2))
intg_crossov_series = np.zeros((trials,ntau,2,2)) # rank, rank
intg_crossovPop_series = np.zeros((trials,ntau,2,2,2)) # rank, rank, population
g, gamma =  6, 1/4.0
NE = int(N/(1+gamma))
NI = N-NE#NE*gamma
N  = NE+NI ### update 
ALPHAE, ALPHAI = NE/N, NI/N
KE, KI = int(Kt/(1+gamma)), int(Kt/(1+gamma)*gamma) ### fixed out-degree
ce, ci = KE/NE, KI/NI
print('ce and ci:',ce,ci)
### assert that the differences between ce and ci are smaller than epsilon
epsilon = 1E-2
assert np.abs(ce-ci)<epsilon
# assert ce==ci
c = ce
ji,je = g*J,J 
### define the network parameters of the diluted Gaussian matrix 
ge, gi = np.sqrt(je**2*c*(1-c)*N), np.sqrt(ji**2*c*(1-c)*N) 
hat_sigmae, hat_sigmai = np.sqrt(c*(1-c)), np.sqrt(c*(1-c))### standard deviation of the adjacency matrix
sigmae,sigmai = np.sqrt(c*(1-c)*J**2*N), np.sqrt(c*(1-c)*(-g*J)**2*N)### with magnitude of the coupling
JE,JI = je*c*NE, ji*c*NI 
lambda0 = JE-JI 
### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 
## TEST THE EIGENVALUES OF THE MEAN MATRIX 
eigvJ0, eigvecJ0 = la.eig(Jbar)
print('eigvJ0:',eigvJ0[0],' theory:',JE-JI)
### mean left and right eigenvectors
leigvec0, reigvec0 = np.zeros((N,N)), np.zeros((N,N))
norm_left = np.zeros(2)
## first eigenvector
leigvec0[:,0], reigvec0[:,0] = nvec[:,0]/(JE-JI)/np.sqrt(N), mvec[:,0]/np.sqrt(N)
norm_left[0]  = la.norm(leigvec0[:,0])
leigvec0[:,0] = leigvec0[:,0]/norm_left[0]
norml0_series[:,0,0] = np.sum(leigvec0[:,0]*reigvec0[:,0])
## second eigenvector
kk = np.sqrt(NE*JI**2+NI*JE**2)
reigvec0[:NE,1], reigvec0[NE:,1] = JI/kk,JE/kk 
leigvec0[:NE,1], leigvec0[NE:,1] = -kk/(JE-JI)/NE,kk/(JE-JI)/NI 
norm_left[1]     = la.norm(leigvec0[:,1])
leigvec0[:,1] = leigvec0[:,1]/norm_left[1]
norml0_series[:,0,1] = np.sum(leigvec0[:,1]*reigvec0[:,1])
outerproduct  = np.sum(leigvec0[:,0]*reigvec0[:,1])#*norm_left
print('JE:',JE,'JI:',JI)

ce and ci: 0.2 0.2
eigvJ0: (-0.3899999999999837+0j)  theory: -0.39000000000000024
JE: 0.7799999999999999 JI: 1.1700000000000002


In [5]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
Ipert = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
# Ipert[NE:]=0
Ipert[:NE]=0
tt = np.linspace(0,200,1000)

In [6]:
### generate i.i.d. s
def randbin(M,N,P):  
    return np.random.choice([0, 1], size=(M,N), p=[P, 1-P])

sparseiid_dyns = np.zeros((trials,N,1))
for ktrial in range(trials):
    ### generate iid sparse connectivity 
    Ecol = randbin(1,NE*N,1-ce)
    Icol = randbin(1,NI*N,1-ci)
    Ecol = np.reshape(Ecol,(N,NE))
    Icol = np.reshape(Icol,(N,NI))
    iidsparse = np.zeros((N,N))
    iidsparse[:,:NE], iidsparse[:,NE:] = Ecol.copy()*J,Icol.copy()*J*(-g)
    xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
    xtemporal = odesimulation(tt, xinit, iidsparse, Inp)
    firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))
    sparseiid_dyns[ktrial,:,:] = firing_rate0.copy()
    
sparseiid_dyns = np.squeeze(sparseiid_dyns)
mean_sparseiid_dyns = np.zeros((trials,2))
for ktrial in range(trials):
    mean_sparseiid_dyns[ktrial,0],mean_sparseiid_dyns[ktrial,1]= np.mean(sparseiid_dyns[ktrial,:NE]),np.mean(sparseiid_dyns[ktrial,NE:])
trialavg_sparseiid_dyns = np.mean(mean_sparseiid_dyns,axis=0)
trialstd_sparseiid_dyns = np.std(mean_sparseiid_dyns,axis=0)
print(trialavg_sparseiid_dyns)

In [7]:
np.mean(eigvchn_series[:,:,1],axis=0)

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j])

In [270]:
np.mean(radius_num_map,axis=0)

array([0.11723926, 0.11572596, 0.11394658, 0.1125296 , 0.1111598 ,
       0.10973642, 0.10818744, 0.10623419, 0.10373062, 0.10134946])

In [11]:
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
firing_ratepert = np.zeros((trials,ntau,N))
temporal_fr = np.zeros((trials,ntau,N,1000))
temporal_perturb_fr = np.zeros((trials,ntau,N,1000))
eiglvec0norm_series= np.zeros((trials,ntau,N,2))
leig0mean_series = np.zeros((trials,ntau,N,2))
leig0pre_series = np.zeros((trials,ntau,N,2))

norm_4rvec_series, norm_4lvec_series = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
norm_4lvec_series_ = np.zeros((trials,ntau,2))
flag_run = True
mode = 'heterogeneous'#'nonneg'#'normal'#
for ktrial in range(trials):
    print('~~~~~~~~~~~trial:',ktrial,'~~~~~~~~~~~~~~~')
    # while (flag_run):  
    while(flag_run): 
        xr      = iidGaussian([0,1/np.sqrt(N)],[N,N])
        chneta  = iidGaussian([0,1/np.sqrt(N)],[N,6])
        xrec    = iidGaussian([0,1/np.sqrt(N)],[N,N])
        ### zscore
        xr   = stats.zscore(xr.flatten())
        xr   = xr*1/np.sqrt(N)
        xr   = np.reshape(xr,(N,N))
        ### zscore
        xrec = stats.zscore(xrec.flatten())
        xrec = xrec*1/np.sqrt(N)
        xrec = np.reshape(xrec,(N,N))
        # ### zscore
        # nsample = 20
        chneta  = iidGaussian([0,1/np.sqrt(N)],[N,6])
        chneta[:,0] = stats.zscore(chneta[:,0])
        chneta[:,0] *=(1/np.sqrt(N))
        
        chneta[:,1] = stats.zscore(chneta[:,1])
        chneta[:,1] *=(1/np.sqrt(N))
        
        chneta[:,2] = stats.zscore(chneta[:,2])
        chneta[:,2] *=(1/np.sqrt(N))  
        if mode == 'heterogeneous':
            ''' heterogeneous '''
            aee,aii = np.sqrt(np.abs(tau_series[-1])),np.sqrt(np.abs(tau_series[-1]))
            # aee,aii = np.sqrt(np.abs(0.12)),np.sqrt(np.abs(0.12))#np.sqrt(np.abs(0.1)),np.sqrt(np.abs(0.1))#
            aei,aie = np.sqrt(np.abs(tau_series[-1])),np.sqrt(np.abs(tau_series[-1]))
            zrowe = np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            zcole = np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            
            zrowi = np.repeat(np.reshape(chneta[:,1],(1,-1)),N,axis=0)
            zcoli = np.repeat(np.reshape(chneta[:,1],(-1,1)),N,axis=1)
            
            
            sgnn = 1
            
            z_ee = np.zeros((NE,NE))
            z_ee = aee*(zcole[:NE,:NE])+aee*(zrowe[:NE,:NE])+aei*(zrowi[:NE,:NE])+np.sqrt(1-aee**2-aee**2-aei**2)*xr[:NE,:NE]
            
            z_ii = np.zeros((NI,NI))
            z_ii = aii*(zcoli[NE:,NE:])+aie*(zrowe[NE:,NE:])+aii*(zrowi[NE:,NE:])+np.sqrt(1-aii**2-aie**2-aii**2)*xr[NE:,NE:]
            
            z_ei = np.zeros((NE,NI))
            z_ei = sgnn*aei*(zcoli[:NE,NE:])+aie*(zrowe[:NE,NE:])+aii*(zrowi[:NE,NE:])+np.sqrt(1-aei**2-aie**2-aii**2)*xr[:NE,NE:]
            
            z_ie = np.zeros((NI,NE))
            z_ie = sgnn*aie*(zcole[NE:,:NE])+aee*(zrowe[NE:,:NE])+aei*(zrowi[NE:,:NE])+np.sqrt(1-aie**2-aee**2-aei**2)*xr[NE:,:NE]
            
            zr = np.zeros((N,N))
            zr[:NE,:NE] = z_ee.copy()
            zr[NE:,NE:] = z_ii.copy()
            zr[:NE,NE:] = z_ei.copy()
            zr[NE:,:NE] = z_ie.copy()
            
            ### E-I matrix 
            zr[:,:NE],zr[:,NE:] = zr[:,:NE]*ge,zr[:,NE:]*(-gi)   
        elif mode == 'normal':
            ''' Normal '''
            tau = tau_series[-1]
            a    = np.sqrt(tau)
            zrow = a*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            zcol = a*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            gammarec = a*xrec-a*xrec.copy().T
            ### adjacency matrix
            zr   = zrow.copy()+zcol.copy()+np.sqrt(1-2*tau)*xr### without considering the reciprocal term
            ### E-I matrix 
            zr[:,:NE],zr[:,NE:] = zr[:,:NE]*ge,zr[:,NE:]*(-gi)   
                    
        elif mode == 'nonneg':
            ''' nonneg '''
            aee,aii = np.sqrt(np.abs(tau_series[-1])),np.sqrt(np.abs(tau_series[-1]))
            # aee,aii = np.sqrt(np.abs(0.12)),np.sqrt(np.abs(0.12))#np.sqrt(np.abs(0.1)),np.sqrt(np.abs(0.1))#
            aei,aie = 0, 0
            zrowe = np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            zcole = np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            
            zrowi = np.repeat(np.reshape(chneta[:,1],(1,-1)),N,axis=0)
            zcoli = np.repeat(np.reshape(chneta[:,1],(-1,1)),N,axis=1)
            
            
            sgnn = 1
            
            z_ee = np.zeros((NE,NE))
            z_ee = aee*(zcole[:NE,:NE])+aee*(zrowe[:NE,:NE])+aei*(zrowi[:NE,:NE])+np.sqrt(1-aee**2-aee**2-aei**2)*xr[:NE,:NE]
            
            z_ii = np.zeros((NI,NI))
            z_ii = aii*(zcoli[NE:,NE:])+aie*(zrowe[NE:,NE:])+aii*(zrowi[NE:,NE:])+np.sqrt(1-aii**2-aie**2-aii**2)*xr[NE:,NE:]
            
            z_ei = np.zeros((NE,NI))
            z_ei = sgnn*aei*(zcoli[:NE,NE:])+aie*(zrowe[:NE,NE:])+aii*(zrowi[:NE,NE:])+np.sqrt(1-aei**2-aie**2-aii**2)*xr[:NE,NE:]
            
            z_ie = np.zeros((NI,NE))
            z_ie = sgnn*aie*(zcole[NE:,:NE])+aee*(zrowe[NE:,:NE])+aei*(zrowi[NE:,:NE])+np.sqrt(1-aie**2-aee**2-aei**2)*xr[NE:,:NE]
            
            zr = np.zeros((N,N))
            zr[:NE,:NE] = z_ee.copy()
            zr[NE:,NE:] = z_ii.copy()
            zr[:NE,NE:] = z_ei.copy()
            zr[NE:,:NE] = z_ie.copy()
            
            ### E-I matrix 
            zr[:,:NE],zr[:,NE:] = zr[:,:NE]*ge,zr[:,NE:]*(-gi)  
            
        '''  check the eigenvalues of the matrix '''
        Jchn = Jbar.copy()+zr.copy()
        eigvchn, eigrvec = la.eig(Jchn)
        print(eigvchn[:2])
        if eigvchn[0].imag==0 and eigvchn[1].imag==0:
            flag_run = False
    
    ### ---------------------
    print('Go run...........................')
    intg_ov  = np.zeros(2) ### rank
    intg_ovP = np.zeros((2,2,2)) ### rank, rank, population
    intg_ml, intg_mr = leigvec0.copy(), reigvec0.copy()
    mean_pre = np.array([JE-JI,0])
    mean_total_change = np.zeros(2)
    z_pre = np.zeros((N,N))
    for it, tau in enumerate(tau_series[:]):
        if mode == 'heterogeneous':
            ''' heterogeneous '''
            aee,aii = np.sqrt(np.abs(tau_series[-1])),np.sqrt(np.abs(tau_series[-1]))
            aei,aie = np.sqrt(np.abs(tau)),np.sqrt(np.abs(tau))
            zrowe = np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            zcole = np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            
            zrowi = np.repeat(np.reshape(chneta[:,1],(1,-1)),N,axis=0)
            zcoli = np.repeat(np.reshape(chneta[:,1],(-1,1)),N,axis=1)
            
            if tau<0:
                sgnn = -1
            else:
                sgnn = 1
        
            ### zee and zii    
            z_ee = np.zeros((NE,NE))
            z_ee = aee*(zcole[:NE,:NE])+aee*(zrowe[:NE,:NE])+aei*(zrowi[:NE,:NE])+np.sqrt(1-aee**2-aee**2-aei**2)*xr[:NE,:NE]
            
            z_ii = np.zeros((NI,NI))
            z_ii = aii*(zcoli[NE:,NE:])+aie*(zrowe[NE:,NE:])+aii*(zrowi[NE:,NE:])+np.sqrt(1-aii**2-aie**2-aii**2)*xr[NE:,NE:]
            
            z_ei = np.zeros((NE,NI))
            z_ei = sgnn*aei*(zcoli[:NE,NE:])+aie*(zrowe[:NE,NE:])+aii*(zrowi[:NE,NE:])+np.sqrt(1-aei**2-aie**2-aii**2)*xr[:NE,NE:]
            
            z_ie = np.zeros((NI,NE))
            z_ie = sgnn*aie*(zcole[NE:,:NE])+aee*(zrowe[NE:,:NE])+aei*(zrowi[NE:,:NE])+np.sqrt(1-aie**2-aee**2-aei**2)*xr[NE:,:NE]
            
            zr = np.zeros((N,N))
            zr[:NE,:NE] = z_ee.copy()
            zr[NE:,NE:] = z_ii.copy()
            zr[:NE,NE:] = z_ei.copy()
            zr[NE:,:NE] = z_ie.copy()
            
            ### convergent
            zree, zrei = zr[:NE,:NE].copy(),zr[:NE,NE:].copy()
            conve_ei = np.mean((zrei.T@zree).flatten())/NE*N  
            conve_ee = np.mean((zree.T@zree).flatten())/NE*N
            conve_ii = np.mean((zrei.T@zrei).flatten())/NE*N  
            
            zrii, zrie = zr[NE:,NE:].copy(),zr[NE:,:NE].copy()
            convi_ei = np.mean((zrii.T@zrie).flatten())/NI*N  
            convi_ee = np.mean((zrie.T@zrie).flatten())/NI*N  
            convi_ii = np.mean((zrii.T@zrii).flatten())/NI*N   
            print('conv:',conve_ee,convi_ee,conve_ii,convi_ii,conve_ei,convi_ei)
            print('theo:',aee**2,aie**2,aei**2,aii**2,0,0)
            
            ### divergent 
            divee_e = np.mean((zree@zree.T).flatten())/NE*N 
            divii_e = np.mean((zrie@zrie.T).flatten())/NE*N    
            divei_e = np.mean((zree@zrie.T).flatten())/NE*N  
            
            divee_i = np.mean((zrei@zrei.T).flatten())/NI*N 
            divii_i = np.mean((zrii@zrii.T).flatten())/NI*N 
            divei_i = np.mean((zrei@zrii.T).flatten())/NI*N    
            print('div:',divee_e,divii_e,divei_e,divee_i,divii_i,divei_i)
            print('theo:',aee**2+aei**2,aee**2+aei**2,aee**2+aei**2,aie**2+aii**2,aie**2+aii**2,aie**2+aii**2)
            
            ### reciprocal 
            rec_ee = np.mean(np.diag(zree@zree))/NE*N  
            rec_ii = np.mean(np.diag(zrii@zrii))/NI*N 
            rec_ei = np.mean(np.diag(zrei@zrie))/NI*N   
            rec_ie = np.mean(np.diag(zrie@zrei))/NE*N  
            print('rec:',rec_ee,rec_ii,rec_ei,rec_ie)
            print('theo:',2*aee**2,2*aii**2,aie**2+aei**2,aie**2+aei**2)
        
            ### E-I matrix 
            zr[:,:NE],zr[:,NE:] = zr[:,:NE]*ge,zr[:,NE:]*(-gi)   
        elif mode == 'normal':
            ''' Normal'''
            a    = np.sqrt(tau)
            zrow = a*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            zcol = a*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            gammarec = a*xrec-a*xrec.copy().T
            ### adjacency matrix
            zr   = zrow.copy()+zcol.copy()+np.sqrt(1-2*tau)*xr### without considering the reciprocal term
            ### E-I matrix 
            zr[:,:NE],zr[:,NE:] = zr[:,:NE]*ge,zr[:,NE:]*(-gi) 
        elif mode == 'nonneg':
            ''' nonneg '''
            aee,aii = np.sqrt(np.abs(tau)),np.sqrt(np.abs(tau))
            aei,aie = 0,0
            zrowe = np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            zcole = np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            
            zrowi = np.repeat(np.reshape(chneta[:,1],(1,-1)),N,axis=0)
            zcoli = np.repeat(np.reshape(chneta[:,1],(-1,1)),N,axis=1)
            
            if tau<0:
                sgnn = -1
            else:
                sgnn = 1
        
            ### zee and zii    
            z_ee = np.zeros((NE,NE))
            z_ee = aee*(zcole[:NE,:NE])+aee*(zrowe[:NE,:NE])+aei*(zrowi[:NE,:NE])+np.sqrt(1-aee**2-aee**2-aei**2)*xr[:NE,:NE]
            
            z_ii = np.zeros((NI,NI))
            z_ii = aii*(zcoli[NE:,NE:])+aie*(zrowe[NE:,NE:])+aii*(zrowi[NE:,NE:])+np.sqrt(1-aii**2-aie**2-aii**2)*xr[NE:,NE:]
            
            z_ei = np.zeros((NE,NI))
            z_ei = sgnn*aei*(zcoli[:NE,NE:])+aie*(zrowe[:NE,NE:])+aii*(zrowi[:NE,NE:])+np.sqrt(1-aei**2-aie**2-aii**2)*xr[:NE,NE:]
            
            z_ie = np.zeros((NI,NE))
            z_ie = sgnn*aie*(zcole[NE:,:NE])+aee*(zrowe[NE:,:NE])+aei*(zrowi[NE:,:NE])+np.sqrt(1-aie**2-aee**2-aei**2)*xr[NE:,:NE]
            
            zr = np.zeros((N,N))
            zr[:NE,:NE] = z_ee.copy()
            zr[NE:,NE:] = z_ii.copy()
            zr[:NE,NE:] = z_ei.copy()
            zr[NE:,:NE] = z_ie.copy()
        
            ### E-I matrix 
            zr[:,:NE],zr[:,NE:] = zr[:,:NE]*ge,zr[:,NE:]*(-gi)      
    
        # zr2 = zr@zr   
        # print('tau:',tau)
        # print('zee2:',np.mean((zr[:NE,:NE]@zr[:NE,:NE]).flatten())/NE/(je**2)/c/(1-c))
        # print('zii2:',np.mean((zr[NE:,NE:]@zr[NE:,NE:]).flatten())/NI/(ji**2)/c/(1-c))
        
        # print('zei2:',np.mean((zr[:NE,:NE]@zr[:NE,NE:]).flatten())/NE/(je*ji)/c/(1-c))
        # print('zIE2:',np.mean((zr[:NE,NE:]@zr[NE:,:NE]).flatten())/NI/(ji*je)/c/(1-c))
        

        DELTA_Z = zr-z_pre
        hzr     = DELTA_Z.copy()
        ### generate J connectivity matrix
        Jchn = Jbar.copy()+zr.copy()
        ### full rank simulation
        xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
        xc_temporal = odesimulation(tt, xinit, Jchn, Inp)
        temporal_fr[ktrial,it,:,:] = np.squeeze(xc_temporal.T).copy()
        firing_rateeq[ktrial,it,:] = xc_temporal[-1,:].copy()
        ### perturbation 
        xpert = xc_temporal[-1,:].copy()
        xpert = xpert.reshape(-1,1)
        dtt =tt[1]-tt[0]
        xc_temporal_perturb = []
        for ttt in range(len(tt)):
            delta_x= -xpert + Jchn@xpert.reshape(-1,1)+Ipert.reshape(-1,1)+Inp.reshape(-1,1)
            xpert = delta_x*dtt+xpert 
            xc_temporal_perturb.append(xpert)
        firing_ratepert[ktrial,it,:] = xpert.copy().squeeze()
        temporal_perturb_fr[ktrial,it,:,:] = np.squeeze(np.array(xc_temporal_perturb).T)
        
        
        eigvchn, eigrvec = la.eig(Jchn)
        
        eigvchn_,eiglvec = la.eig(Jchn.copy().T)
        ### normalization
        reig  = np.squeeze(eigrvec[:,:].copy())
        leig0 = np.squeeze(eiglvec[:,:].copy()) 
        normval = np.sum(reig.copy()*leig0.copy(),axis=0)
        norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
        normval = np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        leig    = leig0.copy()/normval.copy() ### left eigenvector normalization
        
        ### sort the eigenvalues by the real part
        if eigvchn[0].real>eigvchn[1].real:
            idxsort = [1,0]
        else:
            idxsort = [0,1]
        eigvchn[:2] = eigvchn[idxsort]
        reig[:,:2] = reig[:,idxsort]
        leig[:,:2] = leig[:,idxsort]
        leig0[:,:2] = leig0[:,idxsort]
        
        if np.mean(reig[:NE,0])*np.mean(reigvec0[:NE,0])<0:
            reig[:,0]*=-1
            leig[:,0]*=-1
            leig0[:,0]*=-1
        if np.mean(reig[:NE,1])<0: ### the second rank-1 component is negative
            reig[:,1]*=-1
            leig[:,1]*=-1
            leig0[:,1]*=-1
            
        ### numerical low-rank approximation 
        ov_inp_lowrank,ov_inp_lowrank_div = np.zeros(2),np.zeros(2)
        vec_lowrank_contribution = np.zeros((N,2))
        ### linear response theory approximation 
        for i in range(2):
            ov_inp_lowrank[i]=np.sum(leig[:,i]*Inp[:])*eigvchn[i]   
            ov_inp_lowrank_div[i]= ov_inp_lowrank[i]/(1-eigvchn[i])
        Equilibrium_lowrank_outliers = np.reshape(Inp.copy(),(N,1))
        # print('EQ shape:',np.shape(Equilibrium_lowrank_outliers))
        for i in range(2):
            vec_lowrank_contribution[:,i] = ov_inp_lowrank_div[i]*reig[:,i]
            # print('shape :...',np.shape(np.reshape(vec_lowrank_contribution[:,i].copy(),(N,1))))
            Equilibrium_lowrank_outliers += np.reshape(vec_lowrank_contribution[:,i].copy(),(N,1))
            #### REDUCE TO 2 POPULATION, THEREFORE RANK, POPULATION
            contributions_lr_num[ktrial,it,i,0] = np.mean(vec_lowrank_contribution[:NE,i])
            contributions_lr_num[ktrial,it,i,1] = np.mean(vec_lowrank_contribution[NE:,i])
        lowrank_eq_num[ktrial,it,:] = np.squeeze(Equilibrium_lowrank_outliers.copy())
        ovs_inplr_num[ktrial,it,:],ovs_inplr_div_num[ktrial,it,:] = ov_inp_lowrank.copy(),ov_inp_lowrank_div.copy() 
            
        ### numerical std 
        intg_std_num_series[ktrial,it,0,0] = np.std(reig[:NE,0])
        intg_std_num_series[ktrial,it,0,1] = np.std(reig[NE:,0])
        intg_std_num_series[ktrial,it,1,0] = np.std(reig[:NE,1])
        intg_std_num_series[ktrial,it,1,1] = np.std(reig[NE:,1])       
        
        
        eigvchn_series[ktrial,it,:]    = eigvchn.copy()#eigvw_norm.copy()#
        eigrvec_series[ktrial,it,:,:]  = reig[:,:2].copy()#eigvecw_norm.copy()#
        eiglvec_series[ktrial,it,:,:]  = leig[:,:2].copy()#eigvect_norm.copy()#
        eiglvec0_series[ktrial,it,:,:] = leig0[:,:2].copy()#eigvect_norm.copy()#
        for iii in range(2):
            eiglvec0norm_series[ktrial,it,:,iii] = leig0[:,iii].copy()/normval[iii,iii]
            
        hzr_u = hzr.copy()
        # hzr_u[:NE,NE:], hzr_u[NE:,:NE]=0,0
        DeltaZ2 = hzr_u@hzr_u    ### used to correct        
        if it<1:
            lvec, rvec = np.squeeze(leigvec0[:,:2]),np.squeeze(reigvec0[:,:2])
            # eigeng     = np.squeeze(eigvchn_series[ktrial,it,:2].copy())
            eigeng     = np.array([JE-JI,0])
            lvec_mean, rvec_mean = lvec.copy(), rvec.copy()
        else:
            lvec, rvec = np.squeeze(eiglvec0_series[ktrial,it-1,:,:2]),np.squeeze(eigrvec_series[ktrial,it-1,:,:2]) ### use the previous eigenvector as the initial condition   
            eigeng = np.squeeze(eigvchn_series[ktrial,it-1,:2].copy())
            ### conditioned mean  
            lvec_mean, rvec_mean = lvec.copy(), rvec.copy()
            lvec_mean[:NE,:],lvec_mean[NE:,:]=np.mean(lvec_mean[:NE,:],axis=0),np.mean(lvec_mean[NE:,:],axis=0)
            rvec_mean[:NE,:],rvec_mean[NE:,:]=np.mean(rvec_mean[:NE,:],axis=0),np.mean(rvec_mean[NE:,:],axis=0)
            
        ### get the appropriate normalization factor
        norm_for_lvec, norm_for_rvec = np.zeros(2),np.zeros(2)
        if it==0:
            hzr_u = xr.copy()
            with_chn = 0
        else:
            hzr_u = hzr.copy()
            with_chn = 1
        
        if mode == 'nonneg':
            truncc = 3
        else:
            truncc = 1
        norm_rvec_temp, norm_lvec_temp = np.zeros((N,2)), np.zeros((N,2))
        for i in range(2):
            rvec_n = np.reshape(rvec[:,i],(-1,1)) + (hzr_u)@np.reshape(rvec[:,i].copy(),(-1,1))/(eigeng[i])#np.real(eigeng[i])
            lvec_n = np.reshape(lvec[:,i],(-1,1)) + (hzr_u).T@np.reshape(lvec[:,i].copy(),(-1,1))/(eigeng[i])#np.real(eigeng[i])

            '''mean connectivity'''
            current_eigv = eigvchn_series[ktrial,it,i].copy()
            if it==0:
                intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
                intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
                leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
                ''' otherwise no correlation can be calculated '''
                norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
                norm_lvec_temp[:,i] = np.squeeze(leig[:,i])*current_eigv
            elif it < truncc and i==1:### alway >0
                ### original values are obtained numerically
                intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
                intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
                leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
                ''' otherwise no correlation can be calculated '''
                norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
                norm_lvec_temp[:,i] = np.squeeze(leig[:,i])*current_eigv
            else:
                eigenvalue_u = (current_eigv)#np.real(current_eigv)
                eigenvalue_um = (eigeng[i]) #np.real(eigeng[i]) 
                
                ### norm_for_rvec and norm_for_lvec 
                rmean_tmp =np.reshape(rvec_mean[:,i].copy(),(-1,1)) + np.reshape(with_chn*(DeltaZ2@np.reshape(rvec_mean[:,i],(-1,1)))/(eigenvalue_um)**2,(N,1))### mean-shifting #np.real(eigenvalue_um)**2,(N,1))### mean-shifting 
                rvec_n[:NE,0] = rvec_n[:NE,0] - np.mean(rvec_n[:NE,0])+rmean_tmp[:NE,0]
                rvec_n[NE:,0] = rvec_n[NE:,0] - np.mean(rvec_n[NE:,0])+rmean_tmp[NE:,0]

                lmean_tmp = np.reshape(lvec_mean[:,i].copy(),(-1,1)) + np.reshape(with_chn*np.reshape(lvec_mean[:,i],(1,-1))@DeltaZ2/(eigeng[i])**2,(N,1))#np.real(eigeng[i])**2,(N,1))### mean-shifting 
                # print('lvec mean E:', lvec_mean[::NE,i])
                if i==1:
                    shiftlvec = np.reshape(with_chn*np.reshape(lvec_mean[:,i],(1,-1))@DeltaZ2/(eigeng[i])**2,(N,1))##np.real(eigeng[i])**2,(N,1))### mean-shifting 

                # print('lvec mean E:', np.mean(lvec_n[:NE,0]),np.mean(lvec_n[NE:,0]))
                lvec_n[:NE,0] = lvec_n[:NE,0] - np.mean(lvec_n[:NE,0])+lmean_tmp[:NE,0]
                lvec_n[NE:,0] = lvec_n[NE:,0] - np.mean(lvec_n[NE:,0])+lmean_tmp[NE:,0]
                
                norm_for_rvec[i] = la.norm(rvec_n) ### normalization factor 
                norm_rvec_n = np.reshape(rvec_n.copy(),(-1,1))/norm_for_rvec[i] 
                norm_rvec_temp[:,i]=np.squeeze(norm_rvec_n.copy())
                
                norm_for_lvec[i] = np.squeeze(np.reshape(lvec_n,(1,-1))@np.reshape(norm_rvec_temp[:,i],(-1,1)))#la.norm(lvec_n)#norml0_series[ktrial,it,i]#
                norm_lvec_n = np.reshape(lvec_n.copy(),(-1,1))/norm_for_lvec[i] 
                # norm_for_lvec[i] = np.squeeze(np.reshape(norm_lvec_n,(1,-1))@np.reshape(norm_rvec_temp[:,i],(-1,1)))       
                # norm_lvec_n = np.reshape(norm_lvec_n.copy(),(-1,1))/np.abs(norm_for_lvec[i])
                ### need to be re-normalized
                norm_lvec_temp[:,i]  = np.squeeze(norm_lvec_n.copy())
                norm_lvec_temp[:,i] = norm_lvec_temp[:,i] *eigvchn_series[ktrial,it,i]#eigvchn_series[ktrial,it,i].real
                
                ### more simplified version using lvec_mean and rvec_mean
                intg_ml[:,i] = lmean_tmp[:,0]/norm_for_lvec[i]*eigenvalue_u
                leig0mean_series[ktrial,it,:,i] =lmean_tmp[:,0]/norm_for_lvec[i]
                leig0pre_series[ktrial,it,:,i] =np.reshape(with_chn*np.reshape(lvec_mean[:,i],(1,-1))@DeltaZ2/(eigeng[i])**2,(N,1))[:,0]##np.real(eigeng[i])**2,(N,1))[:,0]#*eigenvalue_u#
                intg_mr[:,i] = rmean_tmp[:,0]/norm_for_rvec[i]
                
                tilden = np.reshape(lvec_n.copy(),(-1,1))/norml0_series[ktrial,it,i]
                norm_4lvec_series_[ktrial,it,i]=np.squeeze(np.reshape(tilden,(1,-1))@np.reshape(rvec_n.copy(),(-1,1))/norm_for_rvec[i])
                
                norm_4rvec_series[ktrial,it,i],norm_4lvec_series[ktrial,it,i]=norm_for_rvec[i],norm_for_lvec[i]
                # if i==0:
                #     print(i,'should be the same',norm_4lvec_series[ktrial,it,i]/norml0_series[ktrial,it,i],norm_4lvec_series_[ktrial,it,i])
                
            ## @YS 17 Nov, no matter what the variable it is.
            ### modify the mean of the elements on the left and right eigenvectors
            norm_rvec_temp[:NE,i] -= np.mean(norm_rvec_temp[:NE,i])
            intg_std_series[:NE,i] = np.std(norm_rvec_temp[:NE,i]) ### kinda theoretical solutions based on reconstructed eigenvectors
            norm_rvec_temp[NE:,i] -= np.mean(norm_rvec_temp[NE:,i])
            intg_std_series[NE:,i] = np.std(norm_rvec_temp[NE:,i])
            norm_rvec_temp[:NE,i] += np.mean(intg_mr[:NE,i])
            norm_rvec_temp[NE:,i] += np.mean(intg_mr[NE:,i])
            norm_lvec_temp[:NE,i] -= np.mean(norm_lvec_temp[:NE,i])
            norm_lvec_temp[NE:,i] -= np.mean(norm_lvec_temp[NE:,i])
            norm_lvec_temp[:NE,i] += np.mean(intg_ml[:NE,i])
            norm_lvec_temp[NE:,i] += np.mean(intg_ml[NE:,i])
            
            ### also record the reconstructed eigenvectors
            eigrvec_series_rec[ktrial,it,:,i] = norm_rvec_temp[:,i].copy()
            eiglvec_series_rec[ktrial,it,:,i] = norm_lvec_temp[:,i].copy()
            
            intg_mean_series[ktrial,it,:,i,0] = np.reshape(intg_ml[:,i],(N,))
            intg_mean_series[ktrial,it,:,i,1] = np.reshape(intg_mr[:,i],(N,))
            
        
        z_pre = zr.copy()
        flag_run = True
        ### theoretically compute the equilibrium population-averaged firing rate
        an = np.zeros((2,2),dtype=complex)
        am = np.zeros((2,2),dtype=complex) ## population X rank    
        for ir in range(2):
            an[0,ir] = np.mean(intg_ml[:NE,ir])
            an[1,ir] = np.mean(intg_ml[NE:,ir])
            am[0,ir] = np.mean(intg_mr[:NE,ir])
            am[1,ir] = np.mean(intg_mr[NE:,ir])
            
        ### overlap sum
        overlap_inp = np.zeros(2,dtype=complex) 
        for ir in range(2):
            overlap_inp[ir] = (NE*an[0,ir]*Inp[0]+NI*an[1,ir]*Inp[-1])
            ovs_inplr[ktrial,it,ir] = overlap_inp[ir] ### recording_theory
            overlap_inp[ir]/= (1.0-eigvchn[ir])
            ovs_inplr_div[ktrial,it,ir] = overlap_inp[ir] ### recording theory
        eq_fr = np.zeros(2)
        eq_fr[0],eq_fr[1] = Inp[0],Inp[-1]
        for ir in range(2): ### rank two 
            ### record 
            contributions_lr[ktrial,it,ir,0] = overlap_inp[ir]*am[0,ir] ### excitatory population
            contributions_lr[ktrial,it,ir,1] = overlap_inp[ir]*am[1,ir] ### inhibitory population
            eq_fr[0] += am[0,ir]*overlap_inp[ir]
            eq_fr[1] += am[1,ir]*overlap_inp[ir]
        lowrank_eq[ktrial,it,:] = eq_fr.copy() 
         

~~~~~~~~~~~trial: 0 ~~~~~~~~~~~~~~~
[-0.62181644+0.j          0.07786956+0.19963788j]
[-0.66022947+0.j          0.16622941+0.01570387j]
[-0.70515458+0.j          0.05205631+0.14561152j]
[-0.28388406+0.24191239j -0.28388406-0.24191239j]
[-0.75701404+0.j  0.13017024+0.j]
Go run...........................
conv: 0.14815409726889742 0.0006497433719469268 0.004893212804600155 0.1589600511310309 0.000592388078470132 -7.68325778952083e-05
theo: 0.15000000000000002 0.0 0.0 0.15000000000000002 0 0
div: 0.14708256508016723 0.15018077006896854 0.1467364395693789 0.1494765468914424 0.15495071930746288 0.14982981253049635
theo: 0.15000000000000002 0.15000000000000002 0.15000000000000002 0.15000000000000002 0.15000000000000002 0.15000000000000002
rec: 0.29346837914506047 0.3019502238192472 0.0018294313982344839 0.0018294313982344845
theo: 0.30000000000000004 0.30000000000000004 0.0 0.0


C:\Users\Administrator\AppData\Local\Temp\ipykernel_24612\459883251.py:289: ComplexWarning: Casting complex values to real discards the imaginary part
  norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24612\459883251.py:318: ComplexWarning: Casting complex values to real discards the imaginary part
  ov_inp_lowrank[i]=np.sum(leig[:,i]*Inp[:])*eigvchn[i]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24612\459883251.py:319: ComplexWarning: Casting complex values to real discards the imaginary part
  ov_inp_lowrank_div[i]= ov_inp_lowrank[i]/(1-eigvchn[i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24612\459883251.py:323: ComplexWarning: Casting complex values to real discards the imaginary part
  vec_lowrank_contribution[:,i] = ov_inp_lowrank_div[i]*reig[:,i]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24612\459883251.py:340: ComplexWarning: Casting complex values to real di

line 151 eigenvalues: [-1.43694678+0.j  0.87181483+0.j]
conv: 0.1481996550535446 0.0183614111324699 0.02152108021318976 0.15968045722814475 0.002938283076461932 0.0003765841174425611
theo: 0.15000000000000002 0.016666666666666663 0.016666666666666663 0.15000000000000002 0 0
div: 0.16870363965388058 0.171934067094332 0.16846003558339967 0.1680329670762715 0.17518853115547273 0.16909157432238536
theo: 0.16666666666666669 0.16666666666666669 0.16666666666666669 0.16666666666666669 0.16666666666666669 0.16666666666666669
rec: 0.2986335609414966 0.30358231932166274 0.039210619178391956 0.03921061917839196
theo: 0.30000000000000004 0.30000000000000004 0.033333333333333326 0.033333333333333326
line 151 eigenvalues: [-1.38604113+0.j  0.81267809+0.j]
conv: 0.14821394231701376 0.0362480764854896 0.038101566270315516 0.15995078569652646 0.0038960885977209316 0.0005654204516304774
theo: 0.15000000000000002 0.033333333333333326 0.033333333333333326 0.15000000000000002 0 0
div: 0.1874196115712313 0.

C:\Users\Administrator\AppData\Local\Temp\ipykernel_24612\459883251.py:417: ComplexWarning: Casting complex values to real discards the imaginary part
  norm_rvec_temp[:,i]=np.squeeze(norm_rvec_n.copy())
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24612\459883251.py:419: ComplexWarning: Casting complex values to real discards the imaginary part
  norm_for_lvec[i] = np.squeeze(np.reshape(lvec_n,(1,-1))@np.reshape(norm_rvec_temp[:,i],(-1,1)))#la.norm(lvec_n)#norml0_series[ktrial,it,i]#
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24612\459883251.py:424: ComplexWarning: Casting complex values to real discards the imaginary part
  norm_lvec_temp[:,i]  = np.squeeze(norm_lvec_n.copy())
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24612\459883251.py:425: ComplexWarning: Casting complex values to real discards the imaginary part
  norm_lvec_temp[:,i] = norm_lvec_temp[:,i] *eigvchn_series[ktrial,it,i]#eigvchn_series[ktrial,it,i].real
C:\Users\Administrator\AppData\Local\Te

line 151 eigenvalues: [-1.32943271+0.j  0.75195204+0.j]
conv: 0.14822298015176252 0.05416609469168354 0.05467388839445694 0.16014698464243543 0.004624564239890112 0.0007111080046606117
theo: 0.15000000000000002 0.049999999999999996 0.049999999999999996 0.15000000000000002 0 0
div: 0.20565726681389485 0.2087939824957342 0.2054704185056714 0.20450035129475785 0.2128447854512808 0.206093164227913
theo: 0.2 0.2 0.2 0.2 0.2 0.2
rec: 0.3024352243554426 0.30482439540408884 0.11049145004983012 0.11049145004983012
theo: 0.30000000000000004 0.30000000000000004 0.09999999999999999 0.09999999999999999
line 151 eigenvalues: [-1.26893569+0.j  0.68774093+0.j]
conv: 0.14822924122704864 0.07210147844395251 0.07124181269093828 0.16030476297061813 0.005233853734513952 0.0008345934332729738
theo: 0.15000000000000002 0.06666666666666665 0.06666666666666665 0.15000000000000002 0 0
div: 0.2236471886559701 0.22670478659144336 0.22347509082401829 0.22269161379432548 0.23145264036960647 0.22447880934218922
theo

#### transition

In [12]:
(tau_series*c*(1-c)+c**2)/c**2 -1

array([0.        , 0.06666667, 0.13333333, 0.2       , 0.26666667,
       0.33333333, 0.4       , 0.46666667, 0.53333333, 0.6       ])

In [8]:
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/equivalentGaussian/ConnStats_WD_AdjGau_22May_t200ms_PRX_N1500_transition_INHparadoxical.npz"#ConnStats_WD_AdjGau_22May_t200ms_PRX_N1500_nonneg_nonparadoxical_INHparadoxical.npz"
data_name = "/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/equivalentGaussian/ConnStats_WD_AdjGau_22May_t200ms_PRX_N1500_transition_INHparadoxical.npz"#

# np.savez(data_name, **data)
### load data  
data = np.load(data_name,allow_pickle=True)
eigvchn_series = data['eigvchn_series']
eigrvec_series = data['eigrvec_series']
eiglvec_series = data['eiglvec_series']
eiglvec0_series = data['eiglvec0_series']
norml0_series = data['norml0_series']
params = data['params']
intg_ov_series = data['intg_ov_series']
first_perturb_ov = data['first_perturb_ov']
first_perturb_ovP = data['first_perturb_ovP']
intg_mean_series = data['intg_mean_series']
intg_std_series = data['intg_std_series']
intg_std_num_series = data['intg_std_num_series']

firing_rateeq = data['firing_rateeq']
firing_ratepert = data['firing_ratepert']

g = params[()]['g']
gamma = params[()]['gamma']
NE = params[()]['NE']
NI = params[()]['NI']
J = params[()]['J']
ce = params[()]['ce']
ci = params[()]['ci']
tau_series = params[()]['tau_series']

In [9]:
'''case 2'''
tauc = np.abs(tau_series[-1])
zee=J**2*c*(1-c)*NE*tauc*np.ones_like(tau_series) -g*J**2*c*(1-c)*NI*tau_series
zii= -g*J**2*c*(1-c)*NE*tau_series+g**2*J**2*c*(1-c)*NI*tauc*np.ones_like(tau_series) 
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = JE/NE, -JI/NI
def case1eigv(x,J,g,NE,NI,c,tau):
    tauc = np.abs(tau_series[-1])
    zee = J**2*c*(1-c)*NE*tauc-g*J**2*c*(1-c)*NI*tau
    zii = -g*J**2*c*(1-c)*NE*tau+g**2*J**2*c*(1-c)*NI*tauc 
    # print((NE*zee+NI*zii)*4+(JE-JI)**2)
    z2lambda = np.zeros((N,N))
    z2lambda[:,:NE], z2lambda[:,NE:] = zee/x**2,zii/x**2 
    z2lambda = np.eye(N)-z2lambda 
    final = nvec.T@la.inv(z2lambda)@mvec 
    final = np.squeeze(final)
    return final-x

### solve x 
eigvchn_theo = np.zeros((ntau,2))
for it, tau in enumerate(tau_series):
    if it<=-1:
        continue
    x = fsolve(case1eigv,np.mean(eigvchn_series[:,it,1].real),args=(J,g,NE,NI,c,tau))
    eigvchn_theo[it,1] = x 
    print('tau:',tau,'eigv:',x)
    x = fsolve(case1eigv,np.mean((eigvchn_series[:,it,0]).real),args=(J,g,NE,NI,c,tau))
    eigvchn_theo[it,0] = x 
    
    print('tau:',tau,'eigv:',x)
    print('numerical:',np.mean(eigvchn_series[:,it,:2],axis=0))


/var/folders/tc/_48pdxld1ysfyzqxdkycnlzh0000gn/T/ipykernel_2816/2497334240.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigvchn_theo[it,1] = x


tau: 0.0 eigv: [0.91152835]


/var/folders/tc/_48pdxld1ysfyzqxdkycnlzh0000gn/T/ipykernel_2816/2497334240.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigvchn_theo[it,0] = x


tau: 0.0 eigv: [-1.30152835]
numerical: [-1.33461638+0.j  0.92302226+0.j]
tau: 0.016666666666666666 eigv: [0.85510714]
tau: 0.016666666666666666 eigv: [-1.24510714]
numerical: [-1.28042266+0.j  0.87164379+0.j]
tau: 0.03333333333333333 eigv: [0.79547716]
tau: 0.03333333333333333 eigv: [-1.18547716]
numerical: [-1.22348246+0.j  0.81573238+0.j]
tau: 0.05 eigv: [0.73201942]
tau: 0.05 eigv: [-1.12201942]
numerical: [-1.16319753+0.j  0.75613579+0.j]
tau: 0.06666666666666667 eigv: [0.66388591]
tau: 0.06666666666666667 eigv: [-1.05388591]
numerical: [-1.09885832+0.j  0.6922647 +0.j]
tau: 0.08333333333333333 eigv: [0.58985986]
tau: 0.08333333333333333 eigv: [-0.97985986]
numerical: [-1.02947299+0.j  0.62314909+0.j]
tau: 0.1 eigv: [0.5080825]
tau: 0.1 eigv: [-0.8980825]
numerical: [-0.95355506+0.j  0.54726382+0.j]
tau: 0.11666666666666667 eigv: [0.41544656]
tau: 0.11666666666666667 eigv: [-0.80544656]
numerical: [-0.86865331+0.j  0.46196773+0.j]
tau: 0.13333333333333333 eigv: [0.30596407]
tau: 0

In [10]:
### compute the theoretical outliers and the spectral radius
### recording
radius_theo_map = np.zeros(ntau)
### print variables and parameters of the adjacency matrix 
print('-------------------')
print('N:',N)
print('g:',g)
print('gamma:',gamma)
print('J:',J)
print('JE,JI:',JE,JI)
print('ge,gi:',ge,gi)
print('c:',c)
print('je,ji:',je,ji)
print('-------------------')
ce, ci = c,c
for it, tau in enumerate(tau_series):
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([tau,tau])
    tau_con_vec = np.array([[tau,tau],[tau,tau]])
    # tau_rec_vec = np.array([[0,0],[0,0]])
    tau_rec_vec = np.array([[2*tau,2*tau],[2*tau,2*tau]])
    tau_chn_vec = np.array([[tau,tau],[tau,tau]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
    radius_theo_map[it] = radius_theo
            

-------------------
N: 1500
g: 6
gamma: 0.25
J: 0.00325
JE,JI: 0.7799999999999999 1.1700000000000002
ge,gi: 0.05034878350069642 0.3020927010041785
c: 0.2
je,ji: 0.00325 0.0195
-------------------


In [ ]:
# np.shape(xc_temporal_perturb)
# xc_temporal_perturb = np.squeeze(np.array(xc_temporal_perturb))
fig,ax = plt.subplots(2,1,figsize=(6,4),sharex=True,sharey=True)
for ktrial in range(0,trials):
    ax[0].plot(tt,np.mean(temporal_fr[ktrial,-1,NE:,:],axis=0),'green',ls='--',alpha=0.16,lw=2)
    ax[0].plot(tt,np.mean(temporal_perturb_fr[ktrial,-1,NE:,:],axis=0),'orange',alpha=0.16,lw=2)
    
    ax[1].plot(tt,np.mean(temporal_fr[ktrial,0,NE:,:],axis=0),'green',ls='--',alpha=0.6)
    ax[1].plot(tt,np.mean(temporal_perturb_fr[ktrial,0,NE:,:],axis=0),'orange',alpha=0.6)
### set axis 
ax[0].set_xlim(0,100)
# ax[0].set_ylim(0.01,0.2)
ax[0].set_xticks([0,50,100])
ax[0].set_yticks([0.01,0.02])
# ax[0].set_xlabel(r'$t$')
ax[0].set_ylabel('Inhibitory'+r'$r(t)$')
### not right and top lines 
ax[0].spines['right'].set_visible(False)
ax[0].spines['top'].set_visible(False)

### set axis 
ax[1].set_xlim(0,100)
ax[1].set_ylim(0.01,0.02)
ax[1].set_ylim(0.01,1)
ax[1].set_xticks([0,50,100])
ax[1].set_yticks([-0.05,0.05])
ax[1].set_xlabel(r'$t$')
ax[1].set_ylabel('Inhibitory'+r'$r(t)$')
### not right and top lines 
ax[1].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)
plt.show()   

In [11]:
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]#idx#
print('effective trials:',idx_eff,len(idx_eff))

effective trials: [30  3  5 11 10 23 24 19 34 33  2 28  1  9  6  4 14 22  8 17 13 25 32 18
 21 31 12 16  7 15] 30


In [471]:
### compute the numerical radius
# trials = 36
# ntau = 10
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius


In [473]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(6,4))
### label is r_{bulk}^{num}
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

ax.plot(tau_series,eigvchn_theo[:,0],'tab:orange',label='theoretical')
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green')

### plot error bar  
ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='x',color='tab:orange',ecolor='tab:orange',alpha=1,ls='',elinewidth=1.5)

ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=1,ls='',elinewidth=1.5)
ax.plot(tau_series,np.ones_like(eigvchn_series[0,:,0]),'gray',ls='--')
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-1.5,1.0])
ax.set_yticks([-1.5,0,1.0])
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\cbook\__init__.py:1340: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [364]:
### scatter plot the predicted eigenvalues eigvchn_theo against the numerical mean np.mean(eigvchn_series[:,:,0],axis=0)
fig, ax = plt.subplots(figsize=(4,4))
### theoretical eigenvlaues eigvchn_theo against the numerical mean np.mean(eigvchn_series,axis=0)
ax.scatter(np.mean(eigvchn_series[:,:,0],axis=0),eigvchn_theo[:,0],cmap=plt.cm.YlOrRd, c=tau_series, s=50,alpha=1,marker='^')
ax.scatter(np.mean(eigvchn_series[:,:,1],axis=0),eigvchn_theo[:,1],cmap=plt.cm.YlGn, c=tau_series, s=50,alpha=1,marker='^')
### refine the figure
ax.set_xticks([-3,0,3])
ax.set_yticks([-3,0,3])
ax.set_xlim([-3,3])
ax.set_ylim([-3,3])
### set x-axis and y-axis to zero
ax.set_aspect('equal')
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
ax.plot([-3,3],[-3,3],lw=1.5,color='gray',linestyle='--',alpha=0.5)
ax.set_xlabel('mean of numerical')
ax.set_ylabel('theoretical prediction')

d:\UseApp\Anaconda\Lib\site-packages\matplotlib\collections.py:192: ComplexWarning: Casting complex values to real discards the imaginary part
  offsets = np.asanyarray(offsets, float)


Text(0, 0.5, 'theoretical prediction')

In [365]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(4,2))
### label is r_{bulk}^{num}
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0)-np.std(eigvchn_series[idx_eff,:,0].real,axis=0),np.mean(eigvchn_series[:,:,0].real,axis=0)+np.std(eigvchn_series[idx_eff,:,0].real,axis=0),color='tab:red',alpha=0.15)
ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)-np.std(eigvchn_series[idx_eff,:,1].real,axis=0),np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)+np.std(eigvchn_series[idx_eff,:,1].real,axis=0),color='tab:green',alpha=0.15)
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-4.0,2.5])
ax.set_yticks([-3,0,2])
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [366]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(5,3))

ax.plot(tau_series,eigvchn_theo[:,0],'tab:orange',label='theoretical')
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green')

ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='x',color='tab:orange',ecolor='tab:orange',alpha=1,ls='',elinewidth=1.5)
ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=1,ls='',elinewidth=1.5)
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-2.5,1.5])
ax.set_yticks([-2,0,1])
ax.set_ylim([-4.5,3.5])
ax.set_yticks([-4,0,3])

ax.set_ylim([-2.5,1.5])
ax.set_yticks([-2,0,1])
### plot y==1 
ax.plot([0,tau_series[-1]],[1,1],color='gray',linestyle='--',linewidth=1.5)
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [367]:
# ### 
# trials = 36
# ntau = 10
plt.close('all')


In [14]:
### compute the trial averaged mean 
mean_reigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leig0vec_series = np.zeros((trials,ntau,2,2))
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series = np.zeros((trials,ntau,2,2))
thl = 15
ths = 15
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,1],axis=0)
            
            mean_reigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,1],axis=0)
            
            mean_leigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,0],axis=0)
            mean_leigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,0],axis=0)

                
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir])
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir])

            
## select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
            
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        
        

In [373]:
### rescaled sqrt(N)
mean_leigvec_num_series *=np.sqrt(N)
mean_reigvec_num_series *=np.sqrt(N)
mean_leigvec_series *=np.sqrt(N)
mean_reigvec_series *=np.sqrt(N)

In [374]:
dtau = tau_series[1]-tau_series[0]
### plot the mean of the left and right eigenvectors
tau_max = tau_series[-1]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True,sharey=True)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),facecolor='green',alpha=0.3)

### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.2,0.4])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[0].set_ylim(-0.01,0.08)
ax[0].set_yticks([0,0.05])
ax[0].set_ylim(-0.,1.5)
ax[0].set_yticks([0,1.5])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of right \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),facecolor='green',alpha=0.3)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.2,0.4])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of right \n eigenvector(I)',fontsize=12)
ax[1].set_ylim(-0.0,1.5)

fig.tight_layout()


In [375]:
### same plot but for the left eigenvector
dtau = tau_series[1]-tau_series[0]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
idx_eff = np.arange(trials)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### excitatory population
### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
# ax[0].set_xticks([0,0.2,0.4])
# ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[0].set_ylim(-0.2,0.35)
# ax[0].set_yticks([0,0.3])
ax[0].set_xticks([0,0.2,0.4])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[0].set_ylim(-0.05,0.05)
ax[0].set_yticks([0,0.05])
ax[0].set_ylim(-1.2,2.5)
ax[0].set_yticks([-1.0,2])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of left \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
# ax[1].set_xticks([0,0.2,0.4])
# ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[1].set_ylim(-1.5,1)
# ax[1].set_yticks([-1,0])
ax[1].set_xticks([0,0.2,0.4])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[1].set_ylim(-0.05,0.05)
# ax[1].set_yticks([0,0.05])
ax[1].set_ylim(-20,5.)
ax[1].set_yticks([-20,5.])
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of left \n eigenvector(I)',fontsize=12)

d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5398: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[0] = start
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5399: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[N + 1] = end
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5402: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[1:N+1, 1] = dep1slice
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5404: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[N+2:, 1] = dep2slice[::-1]
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\cbook\__init__.py:1340: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


Text(0, 0.5, 'Mean of left \n eigenvector(I)')

In [148]:
### same plot but for the left eigenvector
dtau = tau_series[1]-tau_series[0]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
idx_eff = np.arange(trials)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[0].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,0],axis=0),facecolor='green',alpha=0.3)
## excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[0].set_ylim(-0.01,0.18)
# ax[0].set_yticks([0,0.15])# noparadoxical
ax[0].set_ylim(-0.01,0.06)
ax[0].set_yticks([0,0.05])
ax[0].set_ylim(-0.3,0.1)
ax[0].set_yticks([-0.3,0.1])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of left \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,1],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,1],axis=0),facecolor='green',alpha=0.3)
## move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[1].set_ylim(-0.4,0.02)
ax[1].set_yticks([-0.4,0.])
ax[1].set_ylim(-0.5,1.0)
ax[1].set_yticks([-0.5,1.0])
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of left \n eigenvector(I)',fontsize=12)

Text(0, 0.5, 'Mean of left \n eigenvector(I)')

In [15]:
mean_rvec = np.zeros((ntau,2,2))
mean_lvec = np.zeros((ntau,2,2))

for it in range(ntau):
    for ir in range(2):
        for ip in range(2):
            mean_rvec[it,ir,ip] = np.nanmean(mean_reigvec_num_series[:,it,ir,ip])
            mean_lvec[it,ir,ip] = np.nanmean(mean_leigvec_num_series[:,it,ir,ip])*eigvchn_theo[it,ir].real
            
### compute the theoretical response function 
rank = 2
response_func_contribution = np.zeros((ntau,rank,2))
### compute the theoretical response function 
response_func = np.zeros((trials,ntau,2))
for it in range(ntau):
    response_func[:,it,1] = 1
    if it>=0:
        for ir in range(2):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            # response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))/N
            # response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))/N
            
            # response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))/N
            # response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))/N
            

                
    else:
        for ir in range(1):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            # response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))/N
            # response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))/N
            
            # response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))/N
            # response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))/N


In [16]:
### 
meanfr_eq = np.zeros((trials,ntau,2))
meanfr_pert = np.zeros((trials,ntau,2))
meanfr_eq[:,:,0] = np.mean(firing_rateeq[:,:,:NE],axis=2)
meanfr_eq[:,:,1] = np.mean(firing_rateeq[:,:,NE:],axis=2)
### same for pydll Creates ()
meanfr_pert[:,:,0] = np.mean(firing_ratepert[:,:,:NE],axis=2)
meanfr_pert[:,:,1] = np.mean(firing_ratepert[:,:,NE:],axis=2)
### numerical response function 
response_func_num = np.zeros((trials,ntau,2))
for ktrial in range(trials):
    for it in range(ntau):
        response_func_num[ktrial,it,0]=(meanfr_pert[ktrial,it,0]-meanfr_eq[ktrial,it,0])/Ipert[-1]
        response_func_num[ktrial,it,1]=(meanfr_pert[ktrial,it,1]-meanfr_eq[ktrial,it,1])/Ipert[-1]
        
#### delete the largest 3 and the smallest 3 from response_func_num
for it in range(ntau):
    idxsort = np.where(np.abs(response_func_num[:,it,0].copy())>10)
    idxnan = (idxsort) 
    response_func_num[idxnan,it,0] = np.nan 
    
    
    idxsort = np.where(np.abs(response_func_num[:,it,1].copy())>10)
    idxnan = (idxsort) 
    response_func_num[idxnan,it,1] = np.nan 

In [19]:

### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(tau_series,response_func_contribution[:,0,0],'orange',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_contribution[:,1,0],'green',marker='o',alpha=0.36)

# ax.plot(tau_series,response_func_contribution[:,0,0]+response_func_contribution[:,1,0],'green',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_mean[:,0],'k',marker='o')
### numerical using filled_between 
# ax.fill_between(tau_series,response_func_num_mean[:,0]+np.nanstd(response_func_num[:,:,:],axis=0)[:,0],response_func_num_mean[:,0]-np.nanstd(response_func_num[:,:,:],axis=0)[:,0],color='k',alpha=0.3)
alphass = 0.95
ax.errorbar(tau_series[:],response_func_num_mean[:,0],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,0],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('response function')
ax.set_title('excitatory population')
ax.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
### set yaxis, ylim not too large!
ax.set_ylim([-2,0.2])
ax.set_yticks([-2,0])
ax.set_xticks([0,tau_series[-1]])
ax.set_xlim([0-dtau/8,tau_series[-1]+dtau/8])

plt.show()

In [20]:
### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
figinh, axinh = plt.subplots(figsize=(5,3))
axinh.plot(tau_series,response_func_contribution[:,0,1],'orange',marker='o',alpha=0.36)
axinh.plot(tau_series,response_func_contribution[:,1,1],'green',marker='o',alpha=0.36)
axinh.plot(tau_series,response_func_mean[:,1],'k',marker='o')
alphass = 0.95
axinh.errorbar(tau_series[:],response_func_num_mean[:,1],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,1],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)

# ax.fill_between(tau_series,response_func_num_mean[:,1]+np.std(response_func_num[:,:,:],axis=0)[:,1],response_func_num_mean[:,1]-np.std(response_func_num[:,:,:],axis=0)[:,1],color='black',alpha=0.3)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
axinh.set_xlabel(r'$\tau$')
axinh.set_ylabel('response function')
axinh.set_title('inhibitory population')
axinh.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
axinh.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
axinh.spines['top'].set_color('none')
axinh.spines['right'].set_color('none')
axinh.spines['left'].set_position(('data',0))
## set yaxis, ylim not too large!
# ax.set_ylim([-1,1])
# ax.set_ylim([-1,1.5])
axinh.set_ylim([-1.5,0.5])
axinh.set_xlim([0-dtau/8,tau_series[-1]+dtau/8])
axinh.set_xticks([0,tau_series[-1]])
axinh.set_yticks([-1.5,0.5])

#### sparse homo

In [35]:
### compute the theoretical outliers and the spectral radius
### recording
eigvchn_theo = np.zeros((ntau,2))
radius_theo_map = np.zeros(ntau)
### print variables and parameters of the adjacency matrix 
print('-------------------')
print('N:',N)
print('g:',g)
print('gamma:',gamma)
print('J:',J)
print('JE,JI:',JE,JI)
print('ge,gi:',ge,gi)
print('c:',c)
print('je,ji:',je,ji)
print('-------------------')
ce, ci = c,c
for it, tau in enumerate(tau_series):
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([tau,tau])
    tau_con_vec = np.array([[tau,tau],[tau,tau]])
    # tau_rec_vec = np.array([[0,0],[0,0]])
    tau_rec_vec = np.array([[2*tau,2*tau],[2*tau,2*tau]])
    tau_chn_vec = np.array([[tau,tau],[tau,tau]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
    radius_theo_map[it] = radius_theo
            
### theoretical prediction of the eigenvalues

org_k = J*N*(ALPHAE-g*ALPHAI)/2.0

eigvchn_theo[:,0] = org_k*(c+np.sqrt(c**2+4*c*(1-c)*tau_series))
eigvchn_theo[:,1] = org_k*(c-np.sqrt(c**2+4*c*(1-c)*tau_series))

-------------------
N: 1500
g: 6
gamma: 0.25
J: 0.00325
JE,JI: 0.7799999999999999 1.1700000000000002
ge,gi: 0.05034878350069642 0.3020927010041785
c: 0.2
je,ji: 0.00325 0.0195
-------------------


In [36]:
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]#idx#
print('effective trials:',idx_eff,len(idx_eff))

effective trials: [12 23  5 10 11 18  1 35 14 24  0 26 28 16 27 19 30  2 29  8  4 21 20 13
 17 25  7 34 32 33] 30


In [37]:
### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius


In [458]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(6,4))
### label is r_{bulk}^{num}
ax.plot(tau_series,radius_theo_map,'gray',linestyle='--',label=r'$r_{bulk}^{theo}$')
ax.plot(tau_series,-radius_theo_map,'gray',linestyle='--')
# ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label=r'$r_{bulk}^{num}$')
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

ax.plot(tau_series,eigvchn_theo[:,0],'tab:red',label='theoretical')
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green')

### plot error bar  
ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=1,ls='',elinewidth=1.5)

ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=1,ls='',elinewidth=1.5)
ax.plot((tau_series),(JE-JI)*np.ones_like(tau_series),c='gray')
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim([-1.,0.5])
ax.set_yticks([-1,0,0.5])
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',tau_series[0]))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [441]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(4,2))
### label is r_{bulk}^{num}
# ax.plot(tau_series,radius_theo_map,'gray',linestyle='--',label=r'$r_{bulk}^{theo}$')
# ax.plot(tau_series,-radius_theo_map,'gray',linestyle='--')
# ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label=r'$r_{bulk}^{num}$')
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

### I mean... eigvchn_series[:,:,0].real and eigvchn_series[:,:,1].real compared with eigvchn_theo[:,0] and eigvchn_theo[:,1]
# fig,ax = plt.subplots(figsize=(6,6))
# ax.plot(tau_series,eigvchn_theo[:,0],'tab:red',marker='o',label='theoretical')
# ax.plot(tau_series,eigvchn_theo[:,1],'tab:green',marker='o')
# ax.plot(tau_series,np.mean(eigvchn_series[:,:,0].real,axis=0),'k',label='numerical')
# ax.plot(tau_series,np.mean(eigvchn_series[:,:,1].real,axis=0),'k')
ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0)-np.std(eigvchn_series[idx_eff,:,0].real,axis=0),np.mean(eigvchn_series[:,:,0].real,axis=0)+np.std(eigvchn_series[idx_eff,:,0].real,axis=0),color='tab:red',alpha=0.15)
ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)-np.std(eigvchn_series[idx_eff,:,1].real,axis=0),np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)+np.std(eigvchn_series[idx_eff,:,1].real,axis=0),color='tab:green',alpha=0.15)
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
# ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-4.0,2.5])
ax.set_yticks([-3,0,2])
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [38]:
# ### 
# trials = 36
# ntau = 10
# plt.close('all')
### compute the trial averaged mean 
mean_reigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leig0vec_series = np.zeros((trials,ntau,2,2))
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series = np.zeros((trials,ntau,2,2))
thl = 1.5
ths = 1.5
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,1],axis=0)
            
            mean_reigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,1],axis=0)
            
            mean_leigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,0],axis=0)
            mean_leigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,0],axis=0)
            # mean_leig0vec_series[ktrial,it,ir,0] = np.mean(leig0mean_series[ktrial,it,:NE,ir])
            # mean_leig0vec_series[ktrial,it,ir,1] = np.mean(leig0mean_series[ktrial,it,NE:,ir])
                
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir])
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir])
            mean_leig0vec_num_series[ktrial,it,ir,0] = np.mean(eiglvec0_series[ktrial,it,:NE,ir])
            mean_leig0vec_num_series[ktrial,it,ir,1] = np.mean(eiglvec0_series[ktrial,it,NE:,ir])
            
## select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
    
            
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        
        
        

In [39]:
mean_rvec = np.zeros((ntau,2,2))
mean_lvec = np.zeros((ntau,2,2))

for it in range(ntau):
    for ir in range(2):
        for ip in range(2):
            mean_rvec[it,ir,ip] = np.nanmean(mean_reigvec_num_series[:,it,ir,ip])
            mean_lvec[it,ir,ip] = np.nanmean(mean_leigvec_num_series[:,it,ir,ip])*eigvchn_theo[it,ir]
            
### compute the theoretical response function 
rank = 2
response_func_contribution = np.zeros((ntau,rank,2))
### compute the theoretical response function 
response_func = np.zeros((trials,ntau,2))
it = 0
response_func[:,it,1] = 1
for ir in range(1):
    response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
    response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
    
    response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
    response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)

    
for it in range(1,ntau):
    response_func[:,it,1] = 1
    if eigvchn_theo[it,1]<10:#radius_theo_map[it]:#eigvchn_theo[it-1,1]<radius_theo_map[it-1]:
        # response_func[:,it,:] = np.nan
        # response_func_contribution[it,:,:] = np.nan
        # continue  
        for ir in range(2):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            # response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            # response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            
            # response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            # response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))


    
    else:
        for ir in range(2):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            # response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            # response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            
            # response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            # response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))
        

In [385]:
radius_theo_map

array([0.12737347, 0.12559193, 0.12378476, 0.12195081, 0.12008886,
       0.11819757, 0.11627553, 0.11432118, 0.11233284, 0.11030866])

In [386]:
eigvchn_theo[:,1]

array([0.11942805, 0.13      , 0.14023872, 0.15017387, 0.15983095,
       0.16923207, 0.17839657, 0.18734147, 0.19608183, 0.20463108])

In [45]:
### 
meanfr_eq = np.zeros((trials,ntau,2))
meanfr_pert = np.zeros((trials,ntau,2))
meanfr_eq[:,:,0] = np.mean(firing_rateeq[:,:,:NE],axis=2)
meanfr_eq[:,:,1] = np.mean(firing_rateeq[:,:,NE:],axis=2)
### same for pydll Creates ()
meanfr_pert[:,:,0] = np.mean(firing_ratepert[:,:,:NE],axis=2)
meanfr_pert[:,:,1] = np.mean(firing_ratepert[:,:,NE:],axis=2)
### numerical response function 
response_func_num = np.zeros((trials,ntau,2))
for ktrial in range(trials):
    for it in range(ntau):
        response_func_num[ktrial,it,0]=(meanfr_pert[ktrial,it,0]-meanfr_eq[ktrial,it,0])/Ipert[-1]
        response_func_num[ktrial,it,1]=(meanfr_pert[ktrial,it,1]-meanfr_eq[ktrial,it,1])/Ipert[-1]
        
#### delete the largest 3 and the smallest 3 from response_func_num
for it in range(ntau):
    idxsort = np.where(np.abs(response_func_num[:,it,0].copy())>10)
    idxnan = (idxsort) 
    response_func_num[idxnan,it,0] = np.nan 
    
    
    idxsort = np.where(np.abs(response_func_num[:,it,1].copy())>10)
    idxnan = (idxsort) 
    response_func_num[idxnan,it,1] = np.nan 

In [46]:

### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(tau_series,response_func_contribution[:,0,0],'orange',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_contribution[:,1,0],'green',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_mean[:,0],'k',marker='o')
# ### numerical using filled_between 
# ax.fill_between(tau_series,response_func_num_mean[:,0]+np.nanstd(response_func_num[:,:,:],axis=0)[:,0],response_func_num_mean[:,0]-np.nanstd(response_func_num[:,:,:],axis=0)[:,0],color='k',alpha=0.3)
alphass = 0.95
ax.errorbar(tau_series[:],response_func_num_mean[:,0],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,0],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('response function')
ax.set_title('excitatory population')
ax.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',tau_series[0]))
### set yaxis, ylim not too large!
ax.set_ylim([-2,2])
ax.set_yticks([-2,2])
ax.set_ylim([-1.5,0.5])
ax.set_yticks([-1.5,0.5])
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([tau_series[0],0.15,tau_series[-1]])
plt.show()

In [47]:
### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(tau_series,response_func_contribution[:,0,1],'orange',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_contribution[:,1,1],'green',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_mean[:,1],'k',marker='o')
alphass = 0.95
ax.errorbar(tau_series[:],response_func_num_mean[:,1],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,1],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)

# ax.fill_between(tau_series,response_func_num_mean[:,1]+np.std(response_func_num[:,:,:],axis=0)[:,1],response_func_num_mean[:,1]-np.std(response_func_num[:,:,:],axis=0)[:,1],color='black',alpha=0.3)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('response function')
ax.set_title('inhibitory population')
ax.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',tau_series[0]))
## set yaxis, ylim not too large!
ax.set_ylim([-1,1])
ax.set_ylim([-1,1.5])
ax.set_ylim([-1,0.5])
ax.set_ylim([-1.5,0.5])
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([tau_series[0],0.15,tau_series[-1]])
ax.set_yticks([-1.5,0.5])
plt.show()

#### nonneg

In [23]:
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/equivalentGaussian/ConnStats_WD_AdjGau_22May_t200ms_PRX_N1500_nonneg_nonparadoxical_INHparadoxical.npz"#ConnStats_WD_AdjGau_22May_t200ms_PRX_N1500_nonneg_nonparadoxical_INHparadoxical.npz"
data_name = "/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/equivalentGaussian/ConnStats_WD_AdjGau_22May_t200ms_PRX_N1500_nonneg_nonparadoxical_INHparadoxical.npz"#

# np.savez(data_name, **data)
### load data  
data = np.load(data_name,allow_pickle=True)
eigvchn_series = data['eigvchn_series']
eigrvec_series = data['eigrvec_series']
eiglvec_series = data['eiglvec_series']
eiglvec0_series = data['eiglvec0_series']
norml0_series = data['norml0_series']
params = data['params']
intg_ov_series = data['intg_ov_series']
first_perturb_ov = data['first_perturb_ov']
first_perturb_ovP = data['first_perturb_ovP']
intg_mean_series = data['intg_mean_series']
intg_std_series = data['intg_std_series']
intg_std_num_series = data['intg_std_num_series']

firing_rateeq = data['firing_rateeq']
firing_ratepert = data['firing_ratepert']

g = params[()]['g']
gamma = params[()]['gamma']
NE = params[()]['NE']
NI = params[()]['NI']
J = params[()]['J']
ce = params[()]['ce']
ci = params[()]['ci']
tau_series = params[()]['tau_series']

In [24]:
'''case 2'''
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = JE/NE, -JI/NI
def case1eigv(x,J,g,NE,NI,c,tau):
    tauc = np.abs(tau)
    zee = J**2*c*(1-c)*NE*tauc-g*J**2*c*(1-c)*NI*0
    zii = -g*J**2*c*(1-c)*NE*0+g**2*J**2*c*(1-c)*NI*tauc 
    # print((NE*zee+NI*zii)*4+(JE-JI)**2)
    z2lambda = np.zeros((N,N))
    z2lambda[:,:NE], z2lambda[:,NE:] = zee/x**2,zii/x**2 
    z2lambda = np.eye(N)-z2lambda 
    final = nvec.T@la.inv(z2lambda)@mvec 
    final = np.squeeze(final)
    return final-x

### solve x 
eigvchn_theo = np.zeros((ntau,2))
for it, tau in enumerate(tau_series):
    if it<=-1:
        continue
    x = fsolve(case1eigv,np.mean(eigvchn_series[:,it,1].real),args=(J,g,NE,NI,c,tau))
    eigvchn_theo[it,1] = x 
    print('tau:',tau,'eigv:',x)
    x = fsolve(case1eigv,np.mean((eigvchn_series[:,it,0]).real),args=(J,g,NE,NI,c,tau))
    eigvchn_theo[it,0] = x 
    
    print('tau:',tau,'eigv:',x)
    print('numerical:',np.mean(eigvchn_series[:,it,:2],axis=0))


/var/folders/tc/_48pdxld1ysfyzqxdkycnlzh0000gn/T/ipykernel_2816/3864357377.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigvchn_theo[it,1] = x


tau: 0.0 eigv: [-0.39]


/var/folders/tc/_48pdxld1ysfyzqxdkycnlzh0000gn/T/ipykernel_2816/3864357377.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigvchn_theo[it,0] = x


tau: 0.0 eigv: [-0.39]
numerical: [-0.39192042+0.j         -0.05258488+0.06639072j]
tau: 0.016666666666666666 eigv: [0.21712255]
tau: 0.016666666666666666 eigv: [-0.60712255]
numerical: [-0.61007737+0.j          0.20850289+0.00442929j]
tau: 0.03333333333333333 eigv: [0.35424038]
tau: 0.03333333333333333 eigv: [-0.74424038]
numerical: [-0.74658688+0.j  0.3509781 +0.j]
tau: 0.05 eigv: [0.46339578]
tau: 0.05 eigv: [-0.85339578]
numerical: [-0.85526921+0.j  0.46160471+0.j]
tau: 0.06666666666666667 eigv: [0.55686767]
tau: 0.06666666666666667 eigv: [-0.94686767]
numerical: [-0.94835308+0.j  0.55582599+0.j]
tau: 0.08333333333333333 eigv: [0.63994012]
tau: 0.08333333333333333 eigv: [-1.02994012]
numerical: [-1.03109155+0.j  0.63939922+0.j]
tau: 0.1 eigv: [0.71546417]
tau: 0.1 eigv: [-1.10546417]
numerical: [-1.10631843+0.j  0.7153027 +0.j]
tau: 0.11666666666666667 eigv: [0.78518621]
tau: 0.11666666666666667 eigv: [-1.17518621]
numerical: [-1.17576974+0.j  0.78533333+0.j]
tau: 0.133333333333333

In [25]:
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]#idx#
print('effective trials:',idx_eff,len(idx_eff))

### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius


effective trials: [11 33  7 24 10 21 28 35  8  9 23 34 18  4 14 15  3 13 31 20 25  2 12 30
 27 16 32  1 19  0] 30


In [467]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(6,4))
### label is r_{bulk}^{num}
# ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label=r'$r_{bulk}^{num}$')
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

ax.plot(tau_series,eigvchn_theo[:,0],'tab:orange',label='theoretical')
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green')

### plot error bar  
ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='x',color='tab:orange',ecolor='tab:orange',alpha=1,ls='',elinewidth=1.5)

ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=1,ls='',elinewidth=1.5)
# ax.plot((tau_series),(JE-JI)*np.ones_like(tau_series),c='gray')
ax.plot((tau_series),1*np.ones_like(tau_series),c='gray',ls='--')
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim([-1.5,1.])
ax.set_yticks([-1.5,0,1.])
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',tau_series[0]))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [26]:
# ### 
# trials = 36
# ntau = 10
# plt.close('all')
### compute the trial averaged mean 
mean_reigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leig0vec_series = np.zeros((trials,ntau,2,2))
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series = np.zeros((trials,ntau,2,2))
thl = 1.5
ths = 1.5
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,1],axis=0)
            
            mean_reigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,1],axis=0)
            
            mean_leigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,0],axis=0)
            mean_leigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,0],axis=0)
            # mean_leig0vec_series[ktrial,it,ir,0] = np.mean(leig0mean_series[ktrial,it,:NE,ir])
            # mean_leig0vec_series[ktrial,it,ir,1] = np.mean(leig0mean_series[ktrial,it,NE:,ir])
                
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir])
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir])
            mean_leig0vec_num_series[ktrial,it,ir,0] = np.mean(eiglvec0_series[ktrial,it,:NE,ir])
            mean_leig0vec_num_series[ktrial,it,ir,1] = np.mean(eiglvec0_series[ktrial,it,NE:,ir])
            
## select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
    
            
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        
        
        

In [423]:
### rescaled sqrt(N)
mean_leigvec_num_series *=np.sqrt(N)
mean_reigvec_num_series *=np.sqrt(N)
mean_leigvec_series *=np.sqrt(N)
mean_reigvec_series *=np.sqrt(N)

In [27]:
mean_rvec = np.zeros((ntau,2,2))
mean_lvec = np.zeros((ntau,2,2))

for it in range(ntau):
    for ir in range(2):
        for ip in range(2):
            mean_rvec[it,ir,ip] = np.nanmean(mean_reigvec_num_series[:,it,ir,ip])
            mean_lvec[it,ir,ip] = np.nanmean(mean_leigvec_num_series[:,it,ir,ip])*eigvchn_theo[it,ir]
            
### compute the theoretical response function 
rank = 2
response_func_contribution = np.zeros((ntau,rank,2))
### compute the theoretical response function 
response_func = np.zeros((trials,ntau,2))
it = 0
response_func[:,it,1] = 1
for ir in range(1):
    response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
    response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
    
    response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
    response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)

    
for it in range(1,ntau):
    response_func[:,it,1] = 1
    if eigvchn_theo[it,1]<10:#np.mean(radius_num_map[:,it]):
        # response_func[:,it,:] = np.nan
        # response_func_contribution[it,:,:] = np.nan
        # continue  
        for ir in range(2):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            # response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            # response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            
            # response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            # response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))


    
    else:
        for ir in range(2):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            
            # response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            # response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            
            # response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-np.mean(eigvchn_series[:,it,ir].real))
            # response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-np.mean(eigvchn_series[:,it,ir].real))
        

In [28]:
### 
meanfr_eq = np.zeros((trials,ntau,2))
meanfr_pert = np.zeros((trials,ntau,2))
meanfr_eq[:,:,0] = np.mean(firing_rateeq[:,:,:NE],axis=2)
meanfr_eq[:,:,1] = np.mean(firing_rateeq[:,:,NE:],axis=2)
### same for pydll Creates ()
meanfr_pert[:,:,0] = np.mean(firing_ratepert[:,:,:NE],axis=2)
meanfr_pert[:,:,1] = np.mean(firing_ratepert[:,:,NE:],axis=2)
### numerical response function 
response_func_num = np.zeros((trials,ntau,2))
for ktrial in range(trials):
    for it in range(ntau):
        response_func_num[ktrial,it,0]=(meanfr_pert[ktrial,it,0]-meanfr_eq[ktrial,it,0])/Ipert[-1]
        response_func_num[ktrial,it,1]=(meanfr_pert[ktrial,it,1]-meanfr_eq[ktrial,it,1])/Ipert[-1]
        
#### delete the largest 3 and the smallest 3 from response_func_num
for it in range(ntau):
    idxsort = np.where(np.abs(response_func_num[:,it,0].copy())>10)
    idxnan = (idxsort) 
    response_func_num[idxnan,it,0] = np.nan 
    
    
    idxsort = np.where(np.abs(response_func_num[:,it,1].copy())>10)
    idxnan = (idxsort) 
    response_func_num[idxnan,it,1] = np.nan 

In [31]:

### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(tau_series,response_func_contribution[:,0,0],'orange',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_contribution[:,1,0],'green',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_mean[:,0],'k',marker='o')
# ### numerical using filled_between 
# ax.fill_between(tau_series,response_func_num_mean[:,0]+np.nanstd(response_func_num[:,:,:],axis=0)[:,0],response_func_num_mean[:,0]-np.nanstd(response_func_num[:,:,:],axis=0)[:,0],color='k',alpha=0.3)
alphass = 0.95
ax.errorbar(tau_series[:],response_func_num_mean[:,0],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,0],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('response function')
ax.set_title('excitatory population')
ax.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',tau_series[0]))
### set yaxis, ylim not too large!
ax.set_ylim([-2,2])
ax.set_yticks([-2,2])
ax.set_ylim([-2,0.5])
ax.set_yticks([-2,0.5])
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([tau_series[0],0.15,tau_series[-1]])
plt.show()

In [32]:
### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(tau_series,response_func_contribution[:,0,1],'orange',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_contribution[:,1,1],'green',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_mean[:,1],'k',marker='o')
alphass = 0.95
ax.errorbar(tau_series[:],response_func_num_mean[:,1],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,1],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)

# ax.fill_between(tau_series,response_func_num_mean[:,1]+np.std(response_func_num[:,:,:],axis=0)[:,1],response_func_num_mean[:,1]-np.std(response_func_num[:,:,:],axis=0)[:,1],color='black',alpha=0.3)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('response function')
ax.set_title('inhibitory population')
ax.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',tau_series[0]))
## set yaxis, ylim not too large!
ax.set_ylim([-1,1])
ax.set_ylim([-1,1.5])
ax.set_ylim([-1,0.5])
ax.set_ylim([-1.5,0.5])
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([tau_series[0],0.15,tau_series[-1]])
ax.set_yticks([-1.5,0.5])
plt.show()

#### save data

In [432]:
def list_to_dict(lst, string):
    """
    Transform a list of variables into a dictionary.
    Parameters
    ----------
    lst : list
        list with all variables.
    string : str
        string containing the names, separated by commas.
    Returns
    -------
    d : dict
        dictionary with items in which the keys and the values are specified
        in string and lst values respectively.
    """
    string = string[0]
    string = string.replace(']', '')
    string = string.replace('[', '')
    string = string.replace('\\', '')
    string = string.replace(' ', '')
    string = string.replace('\t', '')
    string = string.replace('\n', '')
    string = string.split(',')
    d = {s: v for s, v in zip(string, lst)}
    return d

In [34]:
# params = {'g':g,
#           'gamma':gamma,
#           'NE':NE,
#           'NI':NI,
#           'J':J,
#           'ce':ce,
#           'ci':ci,
#           'tau_series':tau_series,
#           }
# lst = [eigvchn_series, eigrvec_series, eiglvec_series,
#        eigrvec_series_rec, eiglvec_series_rec,
#         eiglvec0_series, norml0_series, params,
#         lowrank_eq, firing_rateeq,lowrank_eq_num,firing_ratepert,
#         # temporal_fr, temporal_perturb_fr,
#         ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,
#         contributions_lr, contributions_lr_num,
#         intg_ov_series,first_perturb_ov,first_perturb_ovP,
#         intg_mean_series,intg_std_series,intg_std_num_series,
#         intg_crossov_series,intg_crossovPop_series,leig0mean_series,
#         norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_]
# stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
#        "eigrvec_series_rec, eiglvec_series_rec,"
#         "eiglvec0_series, norml0_series, params,"
#         "lowrank_eq, firing_rateeq,lowrank_eq_num,firing_ratepert,"
#         # "temporal_fr, temporal_perturb_fr,"
#         "ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,"
#         "contributions_lr, contributions_lr_num,"
#         "intg_ov_series,first_perturb_ov,first_perturb_ovP,"
#         "intg_mean_series,intg_std_series,intg_std_num_series,"
#         "intg_crossov_series,intg_crossovPop_series,leig0mean_series,"
#         "norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_"]
# data = list_to_dict(lst=lst, string=stg)
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/equivalentGaussian/ConnStats_WD_AdjGau_22May_t200ms_PRX_N1500_homo_nonparadoxical_INHparadoxical.npz"#ConnStats_WD_AdjGau_22May_t200ms_PRX_N1500_nonneg_nonparadoxical_INHparadoxical.npz"
data_name = "/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/equivalentGaussian/ConnStats_WD_AdjGau_22May_t200ms_PRX_N1500_homo_nonparadoxical_INHparadoxical.npz"#

# np.savez(data_name, **data)
### load data  
data = np.load(data_name,allow_pickle=True)
eigvchn_series = data['eigvchn_series']
eigrvec_series = data['eigrvec_series']
eiglvec_series = data['eiglvec_series']
eiglvec0_series = data['eiglvec0_series']
norml0_series = data['norml0_series']
params = data['params']
intg_ov_series = data['intg_ov_series']
first_perturb_ov = data['first_perturb_ov']
first_perturb_ovP = data['first_perturb_ovP']
intg_mean_series = data['intg_mean_series']
intg_std_series = data['intg_std_series']
intg_std_num_series = data['intg_std_num_series']

firing_rateeq = data['firing_rateeq']
firing_ratepert = data['firing_ratepert']

g = params[()]['g']
gamma = params[()]['gamma']
NE = params[()]['NE']
NI = params[()]['NI']
J = params[()]['J']
ce = params[()]['ce']
ci = params[()]['ci']
tau_series = params[()]['tau_series']

In [436]:
g,J

(6, 0.00325)

In [437]:
tau_series

array([0.1       , 0.11111111, 0.12222222, 0.13333333, 0.14444444,
       0.15555556, 0.16666667, 0.17777778, 0.18888889, 0.2       ])